# Pyomo Introduction

#### Content:
[What is Pyomo?](#What-is-Pyomo?)
<br> [A word about mathematical modelling](#A-word-about-mathematical-modelling)
<br> [The Farmer's Choice problem](#The-Farmer's-Choice-problem)
<br> &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;[Mathematical model](#Mathematical-model)
<br> &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;[Pyomo implementation](#Pyomo-implementation)

<hr>

# What is Pyomo?

## Pyomo as a translator

Pyomo is a powerful modelling framework in python. It bridges the gap between how we as modellers think and what the solvers want. Think of __pyomo as a translator__. It takes your code and translates it to a variety of solver interfaces, ranging from the freely available solvers of the coin-or project (awesome guys, check them out <a href="https://www.coin-or.org/" target="_blank">here</a>) to commercial solvers like <a href="http://www.gurobi.com/" target="_blank">gurobi</a>. Because pyomo is a translator and not a solver itself, it is your responsability to install a suitable solver on your system and point pyomo to it. Depending on your problem - Linear (LP), Integer (IP), Mixed-Integer (MIP), Non-Linear (NLP) - I recommend to have a look at these free solvers from coin-or: cbc (LP, IP, MIP) and ipopt (NLP). You can find the cbc solver used in this tutorial in the folder called _Solvers.

## Components of a pyomo model
We start by importing the pyomo package. To build a model in pyomo, we then first create an empty model object. To this model object, we subsequently add __components__ using the dot (.) notation. When we added all necessary components, we choose a solver and let pyomo handle the dirty translation work - et voilà. Commonly used components are:
- Decision variables
- Objectives
- Constraints
- Index sets

On a side note: We will work with a __concrete model__, meaning that we initialize the model with problem specific data as we go. The counterpart would be an abstract model, where the model structure is completely separated from the problem specific data. If you want to know more, have a look at the <a href="http://pyomo.readthedocs.io/en/latest/" target="_blank">documentation</a>.

## Understanding index sets and decision variables
A set is a collection of distinct items, e.g. a collection of strings {"Andi", "Stef", "Karin"} or integers {0, 1, 2, 3}. In pyomo speech, components like decision variables can be indexed over an __index set__ during creation. In other words, we can use an index set to create a decision variable for each item in said set. The set items then act as keys for accessing (indexing) the corresponding decision variable. That works, because decision variables in pyomo are implemented using a dictionary like data structure, whereby the index set item is the key and the actual value of the decision variable (binary, real, etc.) is the corresponding value. 

__To that end, an index set is a set that is used in the context of the creation of a pyomo component (here decision variables), whereby the set items act as keys to access (index) said component.__

That means, we can use the set {"Andi", "Stef", "Karin"} as index set when creating decision variable X. This will result in the creation of 3 distinct decision variables of type X, which can be accessed via X["Andi"], X["Stef"] and X["Karin"]. 

Similarily, if we use the integer set {0, 1, 2, 3} as index set when creating decision variable Y, this will result in 4 decision variables of type Y. They can be accessed via Y[0], Y[1], Y[2], Y[3]. Note how using an integer set starting from 0 can give the impression that Y is actually a 1-dimensional array, which is not true. Lucky us, because the dictionary like implementation is much more flexible.

Pyomo also allows to index a component over multiple index sets. In that case all possible combinations of indices are constructed and the decision variable is accessed much like a dictionary with multi-dimensional key. This allows for a very intuitive representation of hierarchically structured decision variables. Let's look at an example using the two index sets {"Andi", "Stef", "Karin"} and {0, 1, 2, 3}. We may encounter a situation where we need a decision variable for each person, whereby each person has 4 choices. We can easily represent this hierarchical structure in pyomo by indexing decision variable Z over both index sets. This will lead to the creation of 12 (3 names * 4 choices) decision variables of type Z, which can be accessed by Z["name", choice], e.g. Z["Andi",0], Z["Andi",1], Z["Andi",2], Z["Andi",3], Z["Stef",0], Z["Stef",1], etc...

#### It may take some time to get your head around this concept. Just remember this: Properly defining your index sets is key for making your life easy down the road. This is the reason I will also emphasize index sets in the mathematical modelling part as well as in the code.

<hr>

# A word about mathematical modelling

Formulating a mathematical model is worth the effort as it will greatly help you when implementing it in pyomo. You should always first express your thoughts in a clean mathematical notation before diving into the code. Even though it may look scary at first, it will save you a lot of confusion and time in the long run.

To make life easier, I will stick to some conventions:
- I use the capital letters X, Y, Z to denote __decision variables__.
- All other capital letters denote __index sets__.

The following 4 mathematical notations will frequently pop up. Make sure you understand their meaning:

- $I= \{1,2,3\}$ means "I is a set with members 1, 2 and 3." A set is a collection of distinct items.
- $i \in I$ means " i is member of set I".
- $\sum\limits_{i \in I}v_i$ means "use each member i of set I as index (or key) to retrieve data from v and calculate the sum".
- $\forall i \in I$ means "for all members i in set I".

I will structure the mathematical model in the following way:
- Description: A list of all used data and variables with a short description.
- Index sets: Definition of the sets that are used as keys for indexing the decision variables.
- Decision variables: Definition of the decision variables using the index sets.
- Objective: Definition of the objective function.
- Constraints: Definition of the constraints.

<hr>

# The Farmer's Choice problem

#### Category: Linear programming (LP)

## Introduction

A farmer owns 9 ha of land that he uses for growing carrots and grain. He wants to know the amount of area for each crop which maximizes his revenues. Subsidies are given up to an area threshold of 9 ha. Thus, he has no incentive to plant a larger area than the one which can be subsidised. - Slides SAMO Exercise 5

|   Crop            | Subsidies [CHF]  | Max subsidized area [ha] |
|:------------------|:----------------:|:------------------------:|
|   Carrots         | 5'000            | 7                        |
|   Grain           | 8'000            | 4.5                      |

## Mathematical model

#### Description
$
\begin{equation*}
C : \text{Set of crops}\\
X_c : \text{Non-negative real variable indicating how much hectars of land should be dedicated to crop c}\\
p : \text{Price for crops in \$}\\
m : \text{Maximum area up to each crop is subsidized in hectars}\\
a_{tot} : \text{Total available area in hectars}\\
\end{equation*}
$

#### Index set
$
\begin{equation*}
C = \{carrots, grain\}\\
\end{equation*}
$

#### Decision variables
$
\begin{equation*}
X_c \qquad c \in C,\: X \geq 0\\
\end{equation*}
$

#### Objective
$
\begin{equation*}
MAX \: \sum\limits_{c \in C}p_cX_c\\
\end{equation*}
$

#### Constraint: Total available area
$
\begin{equation*}
\sum\limits_{c \in C}X_c \leq a_{tot}\\
\end{equation*}
$

#### Constraint: Maximum subsidized area
$
\begin{equation*}
X_c \leq m_c \qquad \forall \: c \in C \\
\end{equation*}
$

<hr>

## Pyomo implementation

#### Important: Because the following code cells build on each other, you MUST run every code cell starting from now! If you get an error, try selecting the cell and click "Cell" -> "Run All Above" in the taskbar above and then run the cell again.

#### Suggested workflow
1. Load all needed packages and data in your script and transform the data into a suitable structure.
2. Create a model object.
3. Define the index sets.
4. Based on the index sets, define the decision variables.
5. Specify the objective.
6. Specify the constraints.
7. Decide on a suitable solver depending on your problem and solve it.
8. Process the results.

### Step 1: Load all needed packages and data in your script and transform the data into a suitable structure
Import everything from the package pyomo.environ in such a way that we can use it without any prefix:

In [ ]:
from pyomo.environ import *

Specify the path to the solver executable. Note that we use relative paths, whereby .. means one directory up from the current working directory.

In [ ]:
# For windows: r'../_Solvers/Cbc-2.9.9-win32-msvc14/bin/cbc.exe'
# For ubuntu bionic beaver: r'../_Solvers/Ubuntu_Bionic/Cbc-2.9.8/bin/cbc'
solver_path = r'../_Solvers/Cbc-2.9.9-win32-msvc14/bin/cbc.exe'

Here we use dictionaries to hold the prices (p) and maximum subsidized areas (m) of our crops. The total available area (a_tot) is just a number.

In [ ]:
p = {'carrots':5000, 'grain':8000}
m = {'carrots':7, 'grain':4.5}
a_tot = 9

### Step 2: Create a model object
Create a concrete model object and save it as mo:

In [ ]:
mo = ConcreteModel()

### Step 3: Define the index set
$
\begin{equation*}
C = \{carrots, grain\}\\
\end{equation*}
$
<br>
Let's create a __pyomo set component__ and initialize it with a list containing the crops. It is then attached to the model via dot (.) notation. Note how the pyomo components start with capital letters (Set). That is how you can differentiate them from similarly named python objects (e.g. the python set data structure).

Think of __mo.C__ as a normal variable name. Hence whenever we want to make use of the index set, we need to type __mo.C__. We can display the set using the function pprint(), which stands for pretty print.

In [ ]:
mo.C = Set(initialize=['carrots', 'grain'])

In [ ]:
mo.C.pprint()

### Step 4: Based on the index set, define the decision variables
$
\begin{equation*}
X_c \qquad c \in C,\: X \geq 0\\
\end{equation*}
$
<br>
The index set mo.C is used to create a decision variable for each member of the set. Note how easy it is to specify the non-negative real nature and an initialization value.

In [ ]:
mo.X = Var(mo.C, within=NonNegativeReals, initialize=0)

In [ ]:
mo.X.pprint()

### Step 5: Specify the objective
$
\begin{equation*}
MAX \: \sum\limits_{c \in C}p_cX_c\\
\end{equation*}
$
<br>
The objective is defined with an Objective component. The sense keyword defines the direction of the optimization (defaults to minimize). The objective function itself is specified with the expr keyword. Note how strongly the pyomo code resembles the mathematical notation. We use a list comprehension like construct to iterate over all members of the set mo.C ("carrots", "grain") and use them as keys for p and mo.X. The component is attached to mo with the name obj (though, as always, you can use any name you want) and can thus be accessed using mo.obj.

__It is important to understand, that this expression is not a normal line of code that is executed immediately.__ The expression will be executed by the solver during optimization with different values of mo.X to calculate the value of the objective function. The same applies to the expressions in the upcoming constraint.

In [ ]:
mo.obj = Objective(sense=maximize,
                    expr=sum(p[c]*mo.X[c] for c in mo.C))

In [ ]:
mo.obj.pprint()

### Step 6: Specify the constraints

#### Constraint: Total available area
$
\begin{equation*}
\sum\limits_{c \in C}X_c \leq a_{tot}\\
\end{equation*}
$
<br>
Constraints are another pyomo component. They are either defined using the __Constraint component__ or the __ConstraintList component__, which acts as a container for multiple constraints. I usually name constraints starting with a "c_" to indicate a constraints followed by a short description (i.e. c_total_available_area). The actual constraint is defined using the __expr keyword__ similar to the Objective component. Note again, how strongly the pyomo code resembles the mathematical notation.

In [ ]:
mo.c_total_available_area = Constraint(expr=sum(mo.X[c] for c in mo.C) <= a_tot)

In [ ]:
mo.c_total_available_area.pprint()

#### Constraint: Maximum subsidized area
$
\begin{equation*}
X_c \leq m_c \qquad \forall \: c \in C \\
\end{equation*}
$
<br>
This is a constraint that needs to be specified for each crop, as indicated by the $\forall \: c \in C$. This calls for the use of the component __ConstraintList__, a container for Constraint components. Constraints are added to ConstraintList by using ConstraintLists' __.add()__ function, whereby the constraint to be added is specified with the __expr__ keyword.

First, a ConstraintList object is created and added to the model. A for-loop then iterates over all crops in the crop index set and adds a constraint for each.

In [ ]:
mo.c_max_subsidized_area = ConstraintList()
for c in mo.C:
    mo.c_max_subsidized_area.add(expr=mo.X[c] <= m[c])

In [ ]:
mo.c_max_subsidized_area.pprint()

<hr>

### Rule of thumb:
1. Sum symbols $\sum$ in the mathematical notation can be transferred to pyomo code by using sum() and list comprehension like syntax.
2. For-all symbols $\forall$ can be transferred to pyomo code by using a ConstraintList and a for-loop to iterate over the specified set. Multiple $\forall$ symbols require nested for loops.

<hr>

### Step 7: Decide on a suitable solver depending on your problem and solve it
Before starting the optimization it is usually a good thing to check if everything looks good. Luckily pyomo makes this very easy for us. We have used mo.component.pprint() to print information about components, but we can also use mo.pprint() to print a summary of the whole model. The code in the cell below will __write the summary of the model into a file called opti_model.txt in the folder logs__. Take some time to study this file and check it against your mathematical model.

In [ ]:
with open('logs/opti_model.txt', 'w') as f:
    mo.pprint(ostream=f)

If everything looks good, we can go on to solve the model. Pyomo makes this very easy and provides a helper class called SolverFactory. We first create a solver object from it by specifying the name of the solver (here we use cbc as it is a IP problem) and the path where pyomo can find the solver. The path to the solver was defined at the very top of this section and saved to the variable $solver\_path$.
We then pass the model to the solver object. We save a the log of the solver in a file called solver_lot.txt in the folder logs. With the keyword tee=True, the output is also printed. Of particular interest is the information, whether the an optimal solution was found.

In [ ]:
print('--- start solver ---')
solver = SolverFactory('cbc', executable=solver_path)
solver.solve(mo, tee=True, logfile='logs/solver_log.txt')
print('--- finished ---')

### Step 8: Process the results
The last step is to somehow process the results. Here we may write them to an excel file, export them as shape file or - as in this example - simply print them. __To get the value of a pyomo component, we use a function called value().__ 

We get the value of the objective function by using value(mo.obj). Remember that decision variables resemble the dictionary data structure and therefore we can apply a typical construct that is often used to print all key:value pairs of dictionaries. We use the items() function, which returns key:value pairs, which are assigned to the loop variables k and v repectively. To get the value of a decision variable, again use the value() function.

In [ ]:
print('Subsidies: ' + str(value(mo.obj)) + '[$]')
print('-'*20)

for k,v in mo.X.items():
    print(str(k) + ': ' + str(value(v)) + '[ha]')